<a href="https://colab.research.google.com/github/internallylose/Analyzing-tweets-sentimental-analysis-/blob/main/classification_of_sentiments_of_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
#initialising the kaggle lib

In [ ]:
#uploading the json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# uploading the dataset of twitter
#using the api
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#getting the dataset from the zip file
#using zipfile library and ZipFIle module, naming variable twitterdata and giving it the path to zip file
from zipfile import ZipFile
twitterdata = '/content/sentiment140.zip'

#open the file and extract it
with ZipFile(twitterdata, 'r') as file:
  file.extractall()
  print('The twitterdata is extracted')

The twitterdata is extracted


In [ ]:
#adding dependencies and libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#downloading stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords words in english, this is done to reduce the complexity fo the dataset
#print(stopwords.words('english'))

In [ ]:
#converting the data from csv to a dataframe of pandas
dataframe = pd.read_csv('/content/tweets.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
#number of rows and columns
#dataframe.shape

In [ ]:
#printing the head of the dataframe
dataframe.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#giving the heading to columns and reading dataset
column_heading = ['target', 'id', 'date', 'flag', 'user', 'text']
dataframe = pd.read_csv('/content/tweets.1600000.processed.noemoticon.csv', names = column_heading, encoding = 'ISO-8859-1')

In [ ]:
dataframe.shape

(1600000, 6)

In [ ]:
dataframe.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#checking if there are missing values, we either replace those or drop those
dataframe.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the distribution of the target column
dataframe['target'].value_counts() #0 means negative, 4 is positive

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
#changing 4 to 1 as the positive
dataframe.replace({'target':{4:1}}, inplace=True)

In [ ]:
dataframe['target'].value_counts() # 0 means negative, 1 is positive

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
root_word = PorterStemmer()

In [ ]:
#defining the stem function, content = text column or tweets,
def stem(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #not  an alphabet, upper arrow to take only alphabet
  stemmed_content = stemmed_content.lower() #convert everything to lower case
  stemmed_content = stemmed_content.split() #split all words and load them in list
  stemmed_content = [root_word.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
  return ' '.join(stemmed_content)


In [ ]:
#apply this feature to the dataframe
dataframe['stemmed_content'] = dataframe['text'].apply(stem)

In [ ]:
dataframe.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
#print(dataframe['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
#print(dataframe['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#separating data and twee that is tweet and target(0/1)
x = dataframe['stemmed_content'].values
y = dataframe['target'].values

In [ ]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


In [ ]:
#splitting the data to training and test dataframe, we make 4 arrays
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2) #we want almost equal 0 and 1 is meant by stratifying in both training and testing data, random state splits in same way

In [ ]:
print(x.shape, x_train.shape, x_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
#feature extraction using vectorizer by converting textual to numerical data, tfidf vectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train) #fitting the training data to vectorizer function and then transform
x_test = vectorizer.transform(x_test) #only transform

In [ ]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [ ]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

In [ ]:
##Training the ML Logitsic Regression model

In [ ]:
Mlmodel = LogisticRegression(max_iter=1000) #iter == no of time models can go thru the data

In [ ]:
Mlmodel.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# EVALUATION OF MODEL USING ACCURACY SCORE on training data

In [ ]:
x_train_predictions = Mlmodel.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_predictions)

In [ ]:
print('Accuracy score of the training data is ', training_data_accuracy)

Accuracy score of the training data is  0.81018984375


In [ ]:
# EVALUATION OF MODEL USING ACCURACY SCORE on test data

In [ ]:
x_test_predictions = Mlmodel.predict(x_test)
test_data_accuracy = accuracy_score(y_test, x_test_predictions)

In [ ]:
print('Accuracy score of the test data is ', test_data_accuracy)

Accuracy score of the test data is  0.7780375


In [ ]:
#saving the trained model
import pickle

In [ ]:
File_name = 'trained_model.sav'
pickle.dump(Mlmodel, open(File_name, 'wb')) #wb means w write and b means in binary format

In [ ]:
#using saved model for future predictions
loaded_model = pickle.load(open('trained_model.sav', 'rb')) #r means reading

In [ ]:
x_new = x_test[200]
print(y_test[200])

predictions = loaded_model.predict(x_new)
print(predictions)

if (predictions[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [ ]:
x_new = x_test[3]
print(y_test[3])

predictions = loaded_model.predict(x_new)
print(predictions)

if (predictions[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

0
[0]
Negative Tweet
